In [1]:
import tweepy
import json
from dateutil import parser
import time
import os
import ibm_db_dbi
import ibm_db
import subprocess

In [ ]:
#importing file which sets env variable
subprocess.call("./settings.sh", shell = True)

In [2]:
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_hostname = "dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net"            # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_port = "50000"                    # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_uid = os.environ['DSN_ID']
dsn_pwd = os.environ['DSN_PWD']

In [3]:
#Create database connection
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  dnl28708 on host:  dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net


In [ ]:
consumer_key = os.environ['CONSUMER_KEY']
consumer_secret = os.environ['CONSUMER_SECRET']
access_token = os.environ['ACCESS_TOKEN']
access_token_secret = os.environ['ACCESS_TOKEN_SECRET']
password = os.environ['PASSWORD']

def connect(username, created_at, tweet, retweet_count, place , location):
    """
    connect to DB2 database and insert twitter data
    """
    try:
        con = ibm_db.connect(dsn, "", "")
        conn = ibm_db_dbi.Connection(con)   
        cur = conn.cursor()
        #if conn.is_connected():
        """
        Insert twitter data
        """
        # twitter, golf
        query = "INSERT INTO FOOTBALL (username, created_at, tweet, retweet_count,place, location) VALUES (?, ?, ?, ?, ?, ?)"
        cur.execute(query, (username, created_at, tweet, retweet_count, place, location))

    except:
        print(ibm_db.conn_errormsg())

    cur.close()
    conn.close()
    return


# Tweepy class to access Twitter API
class Streamlistener(tweepy.StreamListener):


    def on_connect(self):
        print("You are connected to the Twitter API")


    def on_error(self):
        if status_code != 200:
            print("error found")
            # returning false disconnects the stream
            return False

    """
    This method reads in tweet data as Json
    and extracts the data we want.
    """
    def on_data(self,data):

        try:
            raw_data = json.loads(data)
            #print(raw_data)
            if 'text' in raw_data:
                
                username = raw_data['user']['screen_name']
                created_at = parser.parse(raw_data['created_at'])
                tweet = raw_data['text']
                retweet_count = raw_data['retweet_count']

                if raw_data['place'] is not None:
                    place = raw_data['place']['country']
                    #print(place)
                else:
                    place = None
                location = raw_data['user']['location']
                #insert data just collected into MySQL database
                connect(username, created_at, tweet, retweet_count, place, location)
                print("Tweet colleted at: {} ".format(str(created_at)))
        except:
            print(ibm_db.conn_errormsg())


if __name__== '__main__':

    # # #Allow user input
    # track = []
    # while True:

    # 	input1  = input("what do you want to collect tweets on?: ")
    # 	track.append(input1)

    # 	input2 = input("Do you wish to enter another word? y/n ")
    # 	if input2 == 'n' or input2 == 'N':
    # 		break

    # print("You want to search for {}".format(track))
    # print("Initialising Connection to Twitter API....")
    # time.sleep(2)

    # authentification so we can access twitter
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api =tweepy.API(auth, wait_on_rate_limit=True)

    # create instance of Streamlistener
    listener = Streamlistener(api = api)
    stream = tweepy.Stream(auth, listener = listener)

    track = ['championsleague','manchesterunited','juventus','manchestercity','spurs','ajax']
    #, 'barcelona','manchesterunited','machestercity','spurs','ajax','juventus'
    #track = ['nba', 'cavs', 'celtics', 'basketball']
    # choose what we want to filter by
    stream.filter(track = track, languages = ['en'])

    